# CVX Tutorial
CVXPY is a powerful tool for solving convex optimization problems. 
E.g, the following nonlinear program:

\begin{equation}
  \min_x f(x) \\
  \mathrm{s.t.} \:\: g_i(x) \leq 0 \quad i = 1,...,m \\
  \qquad h_j(x) = 0 \quad j = 1,...,k 
\end{equation}

Where the inequality constraints need not necessarily be $\leq$ and we can also change the $\min$ to a $\max$, but you can convert these constraints and objective functions into whatever form you find easiest to write down and deal with. With all python packages, we must first install and import them



In [1]:
import numpy as np 
import cvxpy as cp

## Basic CVX Functions and Other Essentials

First, let's define a simple optimization problem to solve. Consider the following unconstrained quadratic program
$$\min_x f(x) = \min_x \frac{1}{2}x^TPx + q^Tx + c $$
where we have that $x\in \mathbb{R}^2$ and $P = P^T \succ 0$ or sometimes $P \in \mathbf{S}^n_{++}$ (the group of symmetric positive definite nxn matrices). 

As with all optimization problems written in CVX, we will first begin by defining the variables and constants


In [2]:
n = 10 # number of dimensions of x
x = cp.Variable((n,1)) # cp.Variable(n) creates an optimization variable of length n

#generating a random symmetric pd 
P = np.random.rand(n,n)
P = P.T@P

q = np.random.rand(n,1)
c = np.random.rand()

Where we've just randomly generated some $A$ matrix, $b$ vector, and $c$ from numpy which is conveniently compatible with CVXPY!

Next, let's define the objective function and optimization problem and solve!

In [3]:
f_obj = cp.Minimize(0.5*cp.quad_form(x, P) + q.T@x + c) # cp.Minimize(expr) takes on some scalar expression and defines the objective function that you wish to minimize;
                                             # alternatively, you could use cp.Maximize to maximize.

prob_unconstrained = cp.Problem(f_obj) # cp.Problem(objective, constraints) takes on the objective function we created through cp.Minimize
                         # and constraints which we will define later and creates an optimization problem object. Note that problems are IMMUTABLE
prob_unconstrained.solve() # solve() will obtain the optimal solution if there is one, and will update your optimization variables x.

#printing the variables
print("Optimal objective value from cvx: ", prob_unconstrained.value)
print("Optimal variables from cvx")
print(x.value)
print(prob_unconstrained.status)


Optimal objective value from cvx:  -38.373817110928755
Optimal variables from cvx
[[-108.82117851]
 [  23.99076472]
 [  76.46362569]
 [ -16.22064827]
 [  -5.2245638 ]
 [  27.41788344]
 [  28.63774716]
 [ -37.37050379]
 [ -42.11019536]
 [  33.80132538]]
optimal


From the FONC, we know the solution to this problem must be
$$x^* = -P^{-1}q, \quad f(x^*) = -\frac{1}{2}q^TP^{-1}q + c $$

So the explicit solution is:



In [4]:
x_star = -np.linalg.solve(P,q)

f_star = 0.5*x_star.T@P@x_star + q.T@x_star  + c
# f_star = -0.5*q.T@np.linalg.solve(P,q) + c is the same
print("Optimal objective value: ", f_star)
print("Optimal variables")
print(x_star)

Optimal objective value:  [[-38.37381711]]
Optimal variables
[[-108.82117851]
 [  23.99076472]
 [  76.46362569]
 [ -16.22064827]
 [  -5.2245638 ]
 [  27.41788344]
 [  28.63774716]
 [ -37.37050379]
 [ -42.11019536]
 [  33.80132538]]


Which match as expected

#Constrained QP
Here we will do a quick example on a constrained QP.

Recall that the general form of a QP is
\begin{equation}
  \min_x \frac{1}{2}x^TPx + q^Tx + c \\
  \mathrm{s.t.} \:\: Gx \preceq h \\
  \qquad Ax = b
\end{equation}
Where, $P$ and $q$ are the same as in the unconstrained case, $G \in \mathbb{R}^{m\times n}$ and $h \in \mathbb{R}^m$, $A \in \mathbb{R}^{p\times n}$ and $b \in \mathbb{R}^{p}$ . 

Now let's set it up in CVXPY

In [5]:
# dimensions of constraints
m = 5
p = 5

# ineq constraints
G = np.random.randn(m, n)
h = np.random.randn(m, 1)
# eq constraints
A = np.random.randn(p, n)
b = np.random.randn(p, 1)
# setup optimization problem and solve
f = cp.Minimize(0.5*cp.quad_form(x, P) + q.T@x + c)
prob_constrained = cp.Problem(f, [G@x <= h,
                     A@x == b])
prob_constrained.solve()

#printing the variables
print("Optimal objective value from cvx: ", prob_constrained.value)
print("Optimal variables from cvx")
print(x.value)
print(prob_constrained.status)

Optimal objective value from cvx:  1.1667855759981238
Optimal variables from cvx
[[-0.49401108]
 [ 0.44257943]
 [-0.10495288]
 [ 0.17878906]
 [ 0.85512187]
 [ 0.1214234 ]
 [ 0.74500946]
 [ 0.02242387]
 [-0.59180088]
 [-1.04519222]]
optimal
